# Bachelor 2021

In [5]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [1]:
S = 100
T = 1
r = 0.07
sigma = 0.2
n_simulations = 5
n_steps = 5

In [2]:
dt = T/n_steps

In [4]:
drift = (r-(sigma**2)/2)*dt
a = sigma*np.sqrt(dt)
x = np.random.normal(0,1,(n_simulations, n_steps))